In [117]:
#Download the libraries
import nltk
import re
from sklearn.metrics import confusion_matrix
import pandas as pd
import numpy as np
import csv
import os
import matplotlib.pyplot as plt
import sys
import time
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
from sklearn.metrics import classification_report
from sklearn import tree

Setting working directory

In [118]:
print os.getcwd();

/Users/imacair/Desktop/Products3


In [119]:
os.chdir("/Users/imacair/Desktop/Products3/")

Read the file

In [120]:
data= pd.read_csv('Final_Manual_3006.csv',
                    encoding='latin-1',delimiter=',')

In [121]:
#Converts text into ASCII

In [122]:
#data.text = data.text.str.encode('ascii','replace')

In [123]:
data.message = data.message.str.encode('ascii','replace')

In [124]:
len(data)

4685

In [125]:
#sample_data= data.sample(n=64000)

In [126]:
sample_data= data

In [127]:
#data_t=sample_data["text"]

In [128]:
data_t=sample_data["message"]

In [129]:
len(data_t)

4685

In [130]:
#lowercase
data_t = data_t.str.lower()

In [131]:
data_s=sample_data["sentiment"]

In [132]:
np.unique(data_s)

array([u'1', u'2', u'3', u'4', u'N', u'n'], dtype=object)

In [133]:
#data_s= data_s[~np.isnan(data_s)]

In [134]:
final = data
res5= pd.DataFrame( index=range(0,len(data_t)),columns = {'new_sent'} )
res5[(final.sentiment==u'2')] = '-1'
res5[(final.sentiment==u'1')] = '-1'
res5[(final['sentiment']==u'3')] = '1'
res5[(final['sentiment']==u'4')] = '1'
res5[(final['sentiment']==u'N')] = '0'
res5[(final['sentiment']==u"n")] = '0'
final=pd.concat([final, res5], axis=2)

In [135]:
#final['sentiment'] = final['sentiment'][~pd.isnull(final['sentiment'])]

In [136]:
#final['sentiment']=np.nan_to_num(final['sentiment'])
#final['sentiment'] = final[~np.isnan('sentiment')]
#y = 
#final['sentiment'][~np.isfinite(final['sentiment'])]

In [137]:
np.unique(final.new_sent)

array(['-1', '0', '1'], dtype=object)

Abbriviation translation

In [138]:
with open('abbrev.csv', mode='r') as infile:
    reader = csv.reader(infile)
    replacement = {rows[0].lower():rows[1].lower() for rows in reader              
                  }

In [139]:
#replacement

In [140]:
#replacement = {
##'r':'are',
#'y':'why',
#'u':'you'}

In [141]:
##How in works
s1 = 'y r u l8'

s2 = ' '.join([replacement.get(w, w) for w in s1.split()])
s2

'why? -or- yes are you l8'

In [142]:
result = pd.DataFrame()
result = final

In [143]:
for i in range(len(result)):
    data_t.values[i]=' '.join([replacement.get(w, w) for w in data_t.values[i].split()])

In [144]:
data_t.head(10)

0    medical north, east, west, south today: ankylo...
1    real time -or- retweet @marathoninmabo: the ma...
2       ankylosing spondylitis https://t.co/bngfvsdv6u
3    real time -or- retweet @gmfhx: blog post -- "g...
4    i've had ankylosing spondylitis for 27 years, ...
5    always (in dates) ante pleasure to support the...
6    @nassexercise @gavin_clunie that's the figure ...
7    @briscomhealth https://t.co/iubjnmkcfx. today ...
8    (in dates) ante person world health organizati...
9    where to turn when you have ankylosing spondyl...
Name: message, dtype: object

In [145]:
#lowercase
data_t = data_t.str.lower()
#Remove urls
data_t= data_t.str.replace(r'(http.*) |(http.*)$|\n', "",)
#Remove twitter handles
data_t = data_t.str.replace(r"@\\w+", "")
#remove htmls
data_t = data_t.str.replace(r'<.*?>', "")
#Remove citations
data_t = data_t.str.replace(r'@[a-zA-Z0-9]*', "")

#remove _
#data_t = data_t.str.replace(r'\_+',"")

In [146]:
data_t.head(10)

0    medical north, east, west, south today: ankylo...
1    real time -or- retweet : the mablethorpe marat...
2                              ankylosing spondylitis 
3    real time -or- retweet : blog post -- "gut bac...
4    i've had ankylosing spondylitis for 27 years, ...
5    always (in dates) ante pleasure to support the...
6     _clunie that's the figure for the wider infla...
7                                                  as.
8    (in dates) ante person world health organizati...
9    where to turn when you have ankylosing spondyl...
Name: message, dtype: object

Spliting the data


In [147]:
from sklearn.cross_validation import train_test_split
import numpy as np
from sklearn.model_selection import KFold

In [148]:
data_train, data_test, label_train, label_test = train_test_split(data_t, final.new_sent, test_size=0.3, random_state=2340)

In [149]:
#data_train, data_test, label_train, label_test = train_test_split(data_t, data_s, test_size=0.3, random_state=2340)

In [150]:
t0 = time.time()
vectorizer = TfidfVectorizer(    sublinear_tf=True,
                                 use_idf=True,stop_words = 'english')
train_vectors = vectorizer.fit_transform(data_train)
test_vectors = vectorizer.transform(data_test)
t1 = time.time()
time_vec = t1-t0

In [151]:
print(time_vec)

0.270555019379


In [152]:
# Perform classification with SVM, kernel=linear
classifier_linear = svm.SVC(C=0.6, kernel='linear')
t0 = time.time()
classifier_linear.fit(train_vectors, label_train)
t1 = time.time()
prediction_linear = classifier_linear.predict(test_vectors)
t2 = time.time()
time_linear_train = t1-t0
time_linear_predict = t2-t1

In [153]:
print("Results for SVC(kernel=linear)")
print("Training time: %fs; Prediction time: %fs" % (time_linear_train, time_linear_predict))
print(classification_report(label_test, prediction_linear))
confusion_matrix(label_test, prediction_linear)

Results for SVC(kernel=linear)
Training time: 1.793236s; Prediction time: 0.419931s
             precision    recall  f1-score   support

         -1       0.73      0.67      0.70       391
          0       0.67      0.82      0.74       576
          1       0.73      0.58      0.65       439

avg / total       0.71      0.70      0.70      1406



array([[261,  98,  32],
       [ 44, 472,  60],
       [ 52, 132, 255]])

In [154]:
# Perform classification with SVM, kernel=linear
classifier_liblinear = svm.LinearSVC()
t0 = time.time()
classifier_liblinear.fit(train_vectors, label_train)
t1 = time.time()
prediction_liblinear = classifier_liblinear.predict(test_vectors)
t2 = time.time()
time_liblinear_train = t1-t0
time_liblinear_predict = t2-t1

In [155]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier

In [156]:
# Perform classification with random forest
classifier_rf = RandomForestClassifier()
t0 = time.time()
classifier_rf.fit(train_vectors, label_train)
t1 = time.time()
prediction_rf = classifier_rf.predict(test_vectors)
t2 = time.time()
time_rf_train = t1-t0
time_rf_predict = t2-t1

In [157]:
# Perform classification with Multinomial Naïve Bayes.
classifier_nb = MultinomialNB()
t0 = time.time()
classifier_nb.fit(train_vectors, label_train)
t1 = time.time()
prediction_nb = classifier_nb.predict(test_vectors)
t2 = time.time()
time_nb_train = t1-t0
time_nb_predict = t2-t1

In [158]:

print("Results for SVC(kernel=linear)")
print("Training time: %fs; Prediction time: %fs" % (time_linear_train, time_linear_predict))
print(classification_report(label_test, prediction_linear))
print("Results for LinearSVC()")
print("Training time: %fs; Prediction time: %fs" % (time_liblinear_train, time_liblinear_predict))
print(classification_report(label_test, prediction_liblinear))
print("Results for MultinomialNB()")
print("Training time: %fs; Prediction time: %fs" % (time_nb_train, time_nb_predict))
print(classification_report(label_test, prediction_nb))

Results for SVC(kernel=linear)
Training time: 1.793236s; Prediction time: 0.419931s
             precision    recall  f1-score   support

         -1       0.73      0.67      0.70       391
          0       0.67      0.82      0.74       576
          1       0.73      0.58      0.65       439

avg / total       0.71      0.70      0.70      1406

Results for LinearSVC()
Training time: 0.061244s; Prediction time: 0.000912s
             precision    recall  f1-score   support

         -1       0.70      0.66      0.68       391
          0       0.70      0.74      0.72       576
          1       0.68      0.66      0.67       439

avg / total       0.69      0.69      0.69      1406

Results for MultinomialNB()
Training time: 0.014813s; Prediction time: 0.001256s
             precision    recall  f1-score   support

         -1       0.74      0.55      0.63       391
          0       0.64      0.83      0.72       576
          1       0.68      0.56      0.62       439

avg / to

Here we will try to ensemble

In [159]:
from sklearn.ensemble import VotingClassifier
from sklearn import model_selection
kfold = model_selection.KFold(n_splits=10, random_state=8)

In [160]:
estimators = []
#estimators.append(('rbf_svm', classifier_rbf ))
estimators.append(('linear_svm',classifier_liblinear))
estimators.append(('linear2_svm',classifier_linear))
estimators.append(('random_forest',classifier_rf))
estimators.append(('multi_naive',classifier_nb))
#estimators.append(('lstm',model))

ensemble = VotingClassifier(estimators)

In [161]:

results = model_selection.cross_val_score(ensemble, train_vectors, label_train, cv=kfold)
print(results.mean())

0.68253058104


In [162]:
label_tests = np.asarray(label_test)

In [163]:
label_tests

array(['0', '0', '0', ..., '0', '1', '-1'], dtype=object)

In [164]:
df =[]
df.append('linear_svm')
df.append('linear2_svm')
df.append('random_forest')
df.append('multi_naive')
df.append('label_tests')

In [165]:
prediction_linear

array(['0', '-1', '0', ..., '1', '1', '-1'], dtype=object)

In [166]:
df

['linear_svm', 'linear2_svm', 'random_forest', 'multi_naive', 'label_tests']

In [167]:
df=[prediction_linear, prediction_liblinear, prediction_nb,label_tests]

In [168]:
df = pd.DataFrame(df)
df = df.transpose()
df.columns = ['prediction_linear', 'prediction_liblinear', 'prediction_nb','label_tests']
#df

In [176]:
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from mlxtend.classifier import StackingClassifier
import numpy as np


lr = RandomForestClassifier()
sclf = StackingClassifier(classifiers=[classifier_liblinear,classifier_linear,classifier_nb], 
                          meta_classifier=lr)

print('3-fold cross validation:\n')

for clf, label in zip([classifier_liblinear,classifier_linear, classifier_nb,sclf], 
                      ['linear_svm', 
                       'linear2_svm',
                       'multi_naive',
                       'StackingClassifier']):

    scores = model_selection.cross_val_score(clf, train_vectors, label_train, 
                                              cv=3, scoring='accuracy')
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" 
          % (scores.mean(), scores.std(), label))

3-fold cross validation:

Accuracy: 0.66 (+/- 0.01) [linear_svm]
Accuracy: 0.66 (+/- 0.01) [linear2_svm]
Accuracy: 0.63 (+/- 0.01) [multi_naive]
Accuracy: 0.66 (+/- 0.01) [StackingClassifier]


In [170]:
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from mlxtend.classifier import StackingClassifier
import numpy as np


lr = RandomForestClassifier()
sclf = StackingClassifier(classifiers=[classifier_liblinear,classifier_linear, classifier_nb], 
                          meta_classifier=lr)

print('3-fold cross validation:\n')

for clf, label in zip([classifier_liblinear,classifier_linear,classifier_nb,sclf], 
                      ['linear_svm', 
                       'linear2_svm',
                       'multi_naive',
                       'StackingClassifier']):

    scores = model_selection.cross_val_score(clf, train_vectors, label_train, 
                                              cv=10, scoring='accuracy')
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" 
          % (scores.mean(), scores.std(), label))

3-fold cross validation:

Accuracy: 0.68 (+/- 0.03) [linear_svm]
Accuracy: 0.67 (+/- 0.02) [linear2_svm]
Accuracy: 0.65 (+/- 0.03) [multi_naive]
Accuracy: 0.68 (+/- 0.03) [StackingClassifier]


In [171]:
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from mlxtend.classifier import StackingClassifier
import numpy as np


lr = LogisticRegression()
sclf = StackingClassifier(classifiers=[classifier_liblinear,classifier_linear, classifier_nb], 
                          meta_classifier=lr)

print('3-fold cross validation:\n')

for clf, label in zip([classifier_liblinear,classifier_linear,classifier_nb,sclf], 
                      ['linear_svm', 
                       'linear2_svm',
                       'multi_naive',
                       'StackingClassifier']):

    scores = model_selection.cross_val_score(clf, train_vectors, label_train, 
                                              cv=10, scoring='accuracy')
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" 
          % (scores.mean(), scores.std(), label))

3-fold cross validation:

Accuracy: 0.68 (+/- 0.03) [linear_svm]
Accuracy: 0.67 (+/- 0.02) [linear2_svm]
Accuracy: 0.65 (+/- 0.03) [multi_naive]
Accuracy: 0.68 (+/- 0.03) [StackingClassifier]


In [172]:
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from mlxtend.classifier import StackingClassifier
import numpy as np


lr = LogisticRegression()
sclf = StackingClassifier(classifiers=[classifier_liblinear,classifier_linear, classifier_nb], 
                          meta_classifier=lr)


sclf.fit(train_vectors, label_train)

prediction_sclf = sclf.predict(test_vectors)
confusion_matrix(label_test, prediction_sclf)

array([[257,  84,  50],
       [ 59, 429,  88],
       [ 50,  98, 291]])

In [173]:

sclf = sclf.predict(test_vectors)